# Set Up:
 - Data uppload
 - Yolov5 clone
 - Conversion from png to yolo v5 format
 - Baseline

In [ ]:
import os
import shutil
from PIL import Image
import glob
import random
from google.colab import drive


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5/
!pip install -r requirements.txt
!pip install pycocotools opencv-python

Cloning into 'yolov5'...
remote: Enumerating objects: 17413, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 17413 (delta 64), reused 22 (delta 22), pack-reused 17327 (from 4)
Receiving objects: 100% (17413/17413), 16.29 MiB | 13.64 MiB/s, done.
Resolving deltas: 100% (11929/11929), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s e

In [ ]:

drive.mount('/content/drive')

Mounted at /content/drive


# Unzip files to local

In [7]:

# os.makedirs('/content/cse527/images', exist_ok=True)
# os.makedirs('/content/cse527/labels', exist_ok=True)

# !unzip -q /content/drive/MyDrive/cse527.zip -d /content/
!unzip -q /content/drive/MyDrive/yolov5.zip -d /content/yolov5

# !unzip -q /content/drive/MyDrive/cse527/images/train.zip -d /content/cse527/images
# !unzip -q /content/drive/MyDrive/cse527/images/val.zip -d /content/cse527/images
# !unzip -q /content/drive/MyDrive/cse527/labels/train.zip -d /content/cse527/labels
# !unzip -q /content/drive/MyDrive/cse527/labels/val.zip -d /content/cse527/labels

# Convert lable to txt file

In [ ]:

import os
import json
import cv2
import numpy as np

def cityscapes_to_yolo(cityscapes_annotation_dir, cityscapes_image_dir):
    for split in os.listdir(cityscapes_annotation_dir):
        split_dir = os.path.join(cityscapes_annotation_dir, split)

        if not os.path.isdir(split_dir):
            print(f"Skipping {split} as it's not a directory")
            continue

        # Now iterate through cities in this split
        for city in os.listdir(split_dir):
            city_dir = os.path.join(split_dir, city)

            if not os.path.isdir(city_dir):
                print(f"Skipping {city} as it's not a directory")
                continue

            print(f"\nProcessing city: {city}")

            # Process each JSON annotation file
            json_files_found = 0
            processed_files = 0

            for filename in os.listdir(city_dir):
                if filename.endswith("_gtFine_polygons.json"):
                    json_files_found += 1
                    # Load the annotation file
                    json_path = os.path.join(city_dir, filename)

                    try:
                        with open(json_path, 'r') as f:
                            data = json.load(f)

                        parts = filename.split('_')
                        city_name = parts[0]
                        seq = parts[1]
                        frame = parts[2]

                        # Construct corresponding image path
                        image_name = f"{city_name}_{seq}_{frame}_leftImg8bit.png"
                        image_path = os.path.join(cityscapes_image_dir, split, city, image_name)

                        # Check if image exists
                        if not os.path.exists(image_path):
                            print(f"Warning: Image not found: {image_path}")
                            continue

                        # Read image to get dimensions
                        img = cv2.imread(image_path)
                        if img is None:
                            print(f"Warning: Could not read image: {image_path}")
                            continue

                        h, w, _ = img.shape

                        # Generate YOLO annotations
                        yolo_annotations = []

                        # Updated class ID mapping
                        label_to_class = {
                            'person': 0,  # All human-related classes are assigned class ID 0
                            'rider': 0,
                            'car': 1,     # All vehicle-related classes are assigned class ID 1
                            'truck': 1,
                            'bus': 1,
                            'on rails': 1,
                            'motorcycle': 1,
                            'bicycle': 1,
                            'caravan': 1,
                            'trailer': 1
                        }

                        for obj in data.get('objects', []):
                            label = obj.get('label')

                            if label not in label_to_class:
                                continue  # Skip if the label is not one of the target classes

                            class_id = label_to_class[label]

                            polygon = obj.get('polygon', [])

                            if not polygon:
                                print(f"Warning: Object {label} has no polygon")
                                continue

                            # Extract bounding box from polygon
                            x_coords = [p[0] for p in polygon]
                            y_coords = [p[1] for p in polygon]

                            # Get bounding box coordinates
                            xmin = max(0, min(x_coords))
                            xmax = min(w, max(x_coords))
                            ymin = max(0, min(y_coords))
                            ymax = min(h, max(y_coords))

                            # Skip invalid boxes
                            if xmin >= xmax or ymin >= ymax:
                                print(f"Warning: Invalid bounding box for {label}: xmin={xmin}, xmax={xmax}, ymin={ymin}, ymax={ymax}")
                                continue

                            # Convert to YOLO format (class_id x_center y_center width height)
                            x_center = (xmin + xmax) / (2 * w)
                            y_center = (ymin + ymax) / (2 * h)
                            box_width = (xmax - xmin) / w
                            box_height = (ymax - ymin) / h

                            yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

                        if yolo_annotations:
                            output_file = os.path.join(cityscapes_annotation_dir, split, city, image_name.replace(".png", ".txt"))

                            with open(output_file, 'w') as f:
                                f.write('\n'.join(yolo_annotations))
                            processed_files += 1

                    except Exception as e:
                        print(f"Error processing {json_path}: {str(e)}")

            print(f"\nCity summary: Found {json_files_found} JSON files, processed {processed_files} files")

cityscapes_annotation_dir = "/content/cse527/labels"
cityscapes_image_dir = "/content/cse527/images"

cityscapes_to_yolo(cityscapes_annotation_dir, cityscapes_image_dir)



Processing city: strasbourg

City summary: Found 365 JSON files, processed 361 files

Processing city: hamburg

City summary: Found 248 JSON files, processed 248 files
Skipping hamburg.cache as it's not a directory

Processing city: hanover

City summary: Found 196 JSON files, processed 196 files

Processing city: monchengladbach

City summary: Found 94 JSON files, processed 93 files

Processing city: krefeld

City summary: Found 99 JSON files, processed 99 files
Skipping .DS_Store as it's not a directory

Processing city: munster

City summary: Found 174 JSON files, processed 174 files

Processing city: frankfurt

City summary: Found 267 JSON files, processed 267 files

Processing city: lindau

City summary: Found 59 JSON files, processed 51 files


# Train Baseline

In [ ]:
# prompt: generatel data.yaml

# Create data.yaml file
data_yaml_content = """
train: /content/cse527/images/train
val: /content/cse527/images/val

nc: 2  # Number of classes
names: ['person', 'vehicle']  # Class names
"""

with open('data.yaml', 'w') as f:
  f.write(data_yaml_content)


In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data /content/data.yaml --weights /content/yolov5/runs/train/exp3/weights/best.pt --cache


Streaming output truncated to the last 5000 lines.
      11/49      3.65G     0.0593    0.07444    0.00643        527        640:  95% 60/63 [00:16<00:01,  2.70it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      11/49      3.65G    0.05931    0.07445   0.006435        679        640:  97% 61/63 [00:17<00:00,  2.82it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      11/49      3.65G    0.05934    0.07459   0.006436        742        640:  98% 62/63 [00:17<00:00,  2.97it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      11/49      3.66G    0.05936    

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt --data /content/data.yaml --img 640 --half



val: data=/content/data.yaml, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/cse527/labels/val/frankfurt.cache... 492 images, 8 backgrounds, 0 corrupt: 100% 500/500 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/cse527/images/val/frankfurt/frankfurt_000000_018797_leftImg8bit.png: 1 duplicate labels removed
val: WARNING ⚠️ /content/cse527/images/val/frankfurt/frankfurt_000001_010156_leftImg8bit.png: 1 duplicate labels removed
val: WARNING ⚠️ /content/cse527/images/val/frankfurt/frankfurt_000001_01627

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt \
               --data /content/data.yaml \
               --img 640 \
               --task val \
               --batch 16 \
               --project val \
               --name results


val: data=/content/data.yaml, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=val, name=results, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/cse527/labels/val/frankfurt.cache... 492 images, 8 backgrounds, 0 corrupt: 100% 500/500 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/cse527/images/val/frankfurt/frankfurt_000000_018797_leftImg8bit.png: 1 duplicate labels removed
val: WARNING ⚠️ /content/cse527/images/val/frankfurt/frankfurt_000001_010156_leftImg8bit.png: 1 duplicate labels removed
val: WARNING ⚠️ /content/cse527/images/val/frankfurt/frankfurt_000001_01627

# Test on DAWN Dataset

In [14]:
root_dir = '/content/cse527/dawn'
weather_types = ['Fog', 'Snow', 'Rain', 'Sand']
class_map = {
    1: 0,  # person → people
    2: 0,  # rider → people
    3: 1,  # car → vehicle
    4: 1,  # truck → vehicle
    5: 1,  # bus → vehicle
    6: 1,  # train → vehicle
    7: 1,  # motorcycle → vehicle
    8: 1   # bicycle → vehicle
}

for weather in weather_types:
    label_folder = os.path.join(root_dir, weather, 'labels')
    for filename in os.listdir(label_folder):
        if filename.endswith('.txt'):
            file_path = os.path.join(label_folder, filename)
            with open(file_path, 'r') as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue
                orig_class = int(parts[0])
                if orig_class in class_map:
                    parts[0] = str(class_map[orig_class])
                    new_lines.append(' '.join(parts))

            # Overwrite the file
            with open(file_path, 'w') as f:
                f.write('\n'.join(new_lines) + '\n')

print("Class remapping complete for Fog, Snow, Rain, and Sand.")


Class remapping complete for Fog, Snow, Rain, and Sand.


In [13]:
# Create YAML files for each weather type
weather_conditions = ['Fog', 'Snow', 'Rain', 'Sand']

for weather in weather_conditions:
    data_test_yaml = f"""
    train: /content/cse527/dawn/{weather}
    val: /content/cse527/dawn/{weather}
    test: /content/cse527/dawn/{weather}
    nc: 2  # Number of classes
    names: ['person', 'vehicle']  # Class names
    """
    with open(f'data_test_{weather.lower()}.yaml', 'w') as f:
        f.write(data_test_yaml)


In [ ]:
cd yolov5/

/content/yolov5


In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt \
               --data data_test_fog.yaml \
               --img 640 \
               --task test \
               --project results \
               --name test_fog


val: data=data_test_fog.yaml, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results, name=test_fog, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Fog/labels.cache... 300 images, 1 backgrounds, 0 corrupt: 100% 300/300 [00:00<?, ?it/s]
test: WARNING ⚠️ /content/cse527/dawn/Fog/images/mist-122.jpg: 1 duplicate labels removed
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 10/10 [00:07<00:00,  1.31it/s]
                   all        300       1950      0.734      0.329      0.372      0.234
          

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt \
               --data data_test_rain.yaml \
               --img 640 \
               --task test \
               --project results \
               --name test_rain


val: data=data_test_rain.yaml, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results, name=test_rain, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Rain/labels.cache... 200 images, 0 backgrounds, 0 corrupt: 100% 200/200 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [00:06<00:00,  1.10it/s]
                   all        200       1588      0.763      0.356      0.432      0.238
                person        200         28      0.633      0.429      0.433      0.225
          

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt \
               --data data_test_snow.yaml \
               --img 640 \
               --task test \
               --project results \
               --name test_snow


val: data=data_test_snow.yaml, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results, name=test_snow, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Snow/labels.cache... 204 images, 0 backgrounds, 0 corrupt: 100% 204/204 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [00:06<00:00,  1.01it/s]
                   all        204       2072      0.747      0.392      0.458      0.266
                person        204        159      0.697      0.465      0.496      0.282
          

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt \
               --data data_test_sand_restored.yaml \
               --img 640 \
               --task test \
               --project results \
               --name test_sand


val: data=data_test_sand.yaml, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results, name=test_sand, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Sand/labels.cache... 323 images, 0 backgrounds, 0 corrupt: 100% 323/323 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 11/11 [00:07<00:00,  1.54it/s]
                   all        323       2235      0.733      0.269       0.33      0.198
                person        323        183      0.573      0.295      0.305      0.167
        

# Method 1:

In [ ]:
pip install diffusers transformers accelerate


  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (13.8 MB)
Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline

pipe = StableDiffusionImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
pipe = pipe.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
def apply_weather(image, weather_prompt, strength=0.5):
    result = pipe(prompt=weather_prompt, image=image, strength=strength).images[0]
    return result

In [ ]:
original_train_dir = '/content/cse527/images/train'
original_labels_dir = '/content/cse527/labels/train'

augmented_output_dir = '/content/cse527/augmented'
augmented_images_dir = os.path.join(augmented_output_dir, 'images', 'train')
augmented_labels_dir = os.path.join(augmented_output_dir, 'labels', 'train')

os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_labels_dir, exist_ok=True)

#process images and generate weather variations
def process_with_stable_diffusion(input_dir, output_dir, label_dir, output_label_dir,
                                  weathers=["rain", "fog", "snow", "sand"], limit=100):
    """
    Process images with Stable Diffusion to create weather variations
    and copy corresponding label files, preserving city folder structure
    """
    total_processed = 0

    # Process each city folder
    for city_name in os.listdir(input_dir):
        city_path = os.path.join(input_dir, city_name)

        # Skip if not a directory
        if not os.path.isdir(city_path):
            continue

        print(f"Processing city: {city_name}")

        # Create corresponding output city directories
        city_output_path = os.path.join(output_dir, city_name)
        city_label_output_path = os.path.join(output_label_dir, city_name)

        os.makedirs(city_output_path, exist_ok=True)
        os.makedirs(city_label_output_path, exist_ok=True)

        # Get all images in this city folder
        image_files = glob.glob(os.path.join(city_path, '*.png'))
        if not image_files:
            image_files = glob.glob(os.path.join(city_path, '*.jpg'))

        print(f"  Found {len(image_files)} images in {city_name}")

        # Process a subset if needed
        city_limit = max(3, limit // len(os.listdir(input_dir)))  # Process at least 3 images per city
        if city_limit < len(image_files):
            image_files = image_files[:city_limit]
            print(f"  Processing {city_limit} images from {city_name}")

        # Process each image in the city
        for idx, img_path in enumerate(image_files):
            base_name = os.path.basename(img_path)
            base_name_no_ext = os.path.splitext(base_name)[0]

            print(f"  Processing {idx+1}/{len(image_files)}: {base_name}")

            try:
                # Create weather variations
                image = Image.open(img_path)

                # First, copy the original image and its label
                original_output_path = os.path.join(city_output_path, base_name)
                image.save(original_output_path)

                # Copy the corresponding label file if it exists
                city_label_path = os.path.join(label_dir, city_name)
                label_file = os.path.join(city_label_path, base_name_no_ext + '.txt')

                if os.path.exists(label_file):
                    shutil.copy(label_file, os.path.join(city_label_output_path, base_name_no_ext + '.txt'))
                else:
                    print(f"  Warning: No label file found for {base_name}")
                    continue

                # Generate weather variations
                weather = random.choice(weathers)
                if weather == "rain":
                    prompt = "same cityscape image in heavy rain, raindrops, wet streets, realistic"
                elif weather == "fog":
                    prompt = "same cityscape image in dense fog, misty, low visibility, realistic"
                elif weather == "snow":
                    prompt = "same cityscape image in snow, snowflakes falling, snowy streets, realistic"
                elif weather == "sand":
                    prompt = "same cityscape image in sandstorm, dusty, reduced visibility, realistic"

                #apply the weather effect using Stable Diffusion
                weather_img = apply_weather(image, prompt, strength=0.6)

                #save the weathered image
                weather_output_path = os.path.join(city_output_path, f"{base_name_no_ext}_{weather}.png")
                weather_img.save(weather_output_path)

                #copy the label file for the weather variation (same annotations)
                weather_label_path = os.path.join(city_label_output_path, f"{base_name_no_ext}_{weather}.txt")
                shutil.copy(label_file, weather_label_path)

                total_processed += 1
                if total_processed >= limit:
                    print(f"Reached limit of {limit} total images. Stopping.")
                    return

            except Exception as e:
                print(f"  Error processing {img_path}: {str(e)}")

#val directory structure
augmented_val_images_dir = os.path.join(augmented_output_dir, 'images', 'val')
augmented_val_labels_dir = os.path.join(augmented_output_dir, 'labels', 'val')
os.makedirs(augmented_val_images_dir, exist_ok=True)
os.makedirs(augmented_val_labels_dir, exist_ok=True)

def copy_directory_structure(src_dir, dest_dir):
    """Copy the directory structure and files from src to dest"""
    for item in os.listdir(src_dir):
        src_item = os.path.join(src_dir, item)
        dest_item = os.path.join(dest_dir, item)

        if os.path.isdir(src_item):
            os.makedirs(dest_item, exist_ok=True)
            copy_directory_structure(src_item, dest_item)
        else:
            shutil.copy2(src_item, dest_item)

#copy validation set
print("Copying validation set...")
copy_directory_structure('/content/cse527/images/val', augmented_val_images_dir)
copy_directory_structure('/content/cse527/labels/val', augmented_val_labels_dir)

In [ ]:
import os
augmented_output_dir = '/content/cse527/augmented'
augmented_images_dir = os.path.join(augmented_output_dir, 'images', 'train')
augmented_labels_dir = os.path.join(augmented_output_dir, 'labels', 'train')
augmented_data_yaml = """
train: {}/images/train
val: {}/images/val
test: /content/cse527/dawn

nc: 2  # Number of classes
names: ['person', 'vehicle']  # Class names
""".format(augmented_output_dir, augmented_output_dir)

with open('data_augmented.yaml', 'w') as f:
    f.write(augmented_data_yaml)

In [ ]:
cd yolov5/

/content/yolov5


## Train on augmented data

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data /content/data_augmented.yaml --weights yolov5s.pt


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-05-05 00:00:58.625835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746403258.645323    6062 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746403258.652142    6062 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=/content/data_augmented.yaml, hyp=data/h

# Running detection on test set to see results

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp5/weights/best.pt\
                  --img 640 \
                  --source /content/cse527/dawn/Rain/images/rain_storm-004.jpg\
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp5/weights/best.pt'], source=/content/cse527/dawn/Rain/images/rain_storm-004.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/cse527/dawn/Rain/images/rain_storm-004.jpg: 448x640 1 person, 2 vehicles, 28.3ms
Speed: 0.5ms pre-process, 28.3ms inference, 159.6ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp9
1 labels saved to runs/detect/exp9/labels

In [ ]:
!python detect.py --weights  /content/yolov5/runs/train/exp3/weights/best.pt \
                  --img 640 \
                  --source /content/cse527/dawn/Rain/images/rain_storm-004.jpg\
                  --conf 0.25 \
                  --save-txt \
                  --save-conf

detect: weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], source=/content/cse527/dawn/Rain/images/rain_storm-004.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/cse527/dawn/Rain/images/rain_storm-004.jpg: 448x640 1 vehicle, 29.1ms
Speed: 0.5ms pre-process, 29.1ms inference, 188.7ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp10
1 labels saved to runs/detect/exp10/labels


In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp5/weights/best.pt\
                  --img 640 \
                  --source /content/cse527/dawn/Rain/images/rain_storm-006.jpg\
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp5/weights/best.pt'], source=/content/cse527/dawn/Rain/images/rain_storm-006.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/cse527/dawn/Rain/images/rain_storm-006.jpg: 448x640 1 person, 2 vehicles, 283.8ms
Speed: 2.5ms pre-process, 283.8ms inference, 1.2ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp25
1 labels saved to runs/detect/exp25/labels


In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp3/weights/best.pt\
                  --img 640 \
                  --source /content/cse527/dawn/Rain/images/rain_storm-006.jpg\
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], source=/content/cse527/dawn/Rain/images/rain_storm-006.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/cse527/dawn/Rain/images/rain_storm-006.jpg: 448x640 1 person, 1 vehicle, 280.2ms
Speed: 2.7ms pre-process, 280.2ms inference, 1.2ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp26
1 labels saved to runs/detect/exp26/labels


In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp6/weights/best.pt \
               --data data_test_fog.yaml \
               --img 640 \
               --task test \
               --project results_augumented \
               --name test_fog


val: data=data_test_fog.yaml, weights=['/content/yolov5/runs/train/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results_augumented, name=test_fog, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Fog/labels.cache... 300 images, 1 backgrounds, 0 corrupt: 100% 300/300 [00:00<?, ?it/s]
test: WARNING ⚠️ /content/cse527/dawn/Fog/images/mist-122.jpg: 1 duplicate labels removed
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 10/10 [00:06<00:00,  1.52it/s]
                   all        300       1950      0.675       0.29      0.331      0.175

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp6/weights/best.pt \
               --data data_test_rain.yaml \
               --img 640 \
               --task test \
               --project results_augumented \
               --name test_fog


val: data=data_test_rain.yaml, weights=['/content/yolov5/runs/train/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results_augumented, name=test_fog, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Rain/labels.cache... 200 images, 0 backgrounds, 0 corrupt: 100% 200/200 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [00:06<00:00,  1.10it/s]
                   all        200       1588      0.734      0.391      0.446      0.233
                person        200         28      0.594        0.5      0.497      0.252


In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp6/weights/best.pt \
               --data data_test_sand.yaml \
               --img 640 \
               --task test \
               --project results_augumented \
               --name test_fog


val: data=data_test_sand.yaml, weights=['/content/yolov5/runs/train/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results_augumented, name=test_fog, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Sand/labels.cache... 323 images, 0 backgrounds, 0 corrupt: 100% 323/323 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 11/11 [00:07<00:00,  1.41it/s]
                   all        323       2235      0.603      0.214      0.257      0.132
                person        323        183      0.476      0.246      0.265      0.11

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp6/weights/best.pt \
               --data data_test_snow.yaml \
               --img 640 \
               --task test \
               --project results_augumented \
               --name test_fog


val: data=data_test_snow.yaml, weights=['/content/yolov5/runs/train/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results_augumented, name=test_fog, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Snow/labels.cache... 204 images, 0 backgrounds, 0 corrupt: 100% 204/204 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [00:07<00:00,  1.03s/it]
                   all        204       2072       0.68      0.375      0.424      0.228
                person        204        159      0.635      0.428      0.445      0.229


In [ ]:
# # prompt: unzip /content/drive/MyDrive/cse527.zip

# !unzip -q /content/drive/MyDrive/cse527.zip -d /content/
# !unzip -q /content/drive/MyDrive/yolov5.zip -d /content/


In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp5/weights/best.pt \
                  --img 640 \
                  --source /content/cse527/dawn/Snow/images/snow_storm-009.jpg \
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp5/weights/best.pt'], source=/content/cse527/dawn/Snow/images/snow_storm-009.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/cse527/dawn/Snow/images/snow_storm-009.jpg: 448x640 7 vehicles, 508.1ms
Speed: 3.9ms pre-process, 508.1ms inference, 2.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp29
1 labels saved to runs/detect/exp29/labels


In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp3/weights/best.pt\
                  --img 640 \
                  --source /content/cse527/dawn/Snow/images/snow_storm-009.jpg\
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], source=/content/cse527/dawn/Snow/images/snow_storm-009.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/cse527/dawn/Snow/images/snow_storm-009.jpg: 448x640 2 persons, 6 vehicles, 286.2ms
Speed: 2.4ms pre-process, 286.2ms inference, 1.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp30
1 labels saved to runs/detect/exp30/labels


In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp3/weights/best.pt\
                  --img 640 \
                  --source /content/cse527/dawn/Sand/images/dusttornado-002.jpg\
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], source=/content/cse527/dawn/Sand/images/dusttornado-002.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/cse527/dawn/Sand/images/dusttornado-002.jpg: 512x640 6 vehicles, 406.2ms
Speed: 4.2ms pre-process, 406.2ms inference, 1.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp36
1 labels saved to runs/detect/exp36/labels


In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp5/weights/best.pt\
                  --img 640 \
                  --source /content/cse527/dawn/Sand/images/dusttornado-002.jpg\
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp5/weights/best.pt'], source=/content/cse527/dawn/Sand/images/dusttornado-002.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/cse527/dawn/Sand/images/dusttornado-002.jpg: 512x640 2 vehicles, 344.7ms
Speed: 2.8ms pre-process, 344.7ms inference, 1.2ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp37
1 labels saved to runs/detect/exp37/labels


# Method 2: WeatherDiff Restoration


In [5]:
!git clone https://github.com/IGITUGraz/WeatherDiffusion.git
%cd WeatherDiffusion


Cloning into 'WeatherDiffusion'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 77 (delta 34), reused 44 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 26.77 KiB | 8.92 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/content/WeatherDiffusion


In [17]:
# Create YAML files for each weather type
weather_conditions = ['Fog', 'Snow', 'Rain', 'Sand']

for weather in weather_conditions:
    data_test_yaml = f"""
    train: /content/cse527/dawn/{weather}
    val: /content/cse527/dawn/{weather}
    test: /content/cse527/restored_images/{weather}
    nc: 2  # Number of classes
    names: ['person', 'vehicle']  # Class names
    """
    with open(f'data_test_{weather.lower()}.yaml', 'w') as f:
        f.write(data_test_yaml)


In [29]:
# Simple script to restore all weather-affected test images
import os
import glob
import cv2
from tqdm import tqdm

# Define the paths
base_path = '/content/cse527/dawn'
weather_folders = ['Fog', 'Rain', 'Snow', 'Sand']

#output directory
os.makedirs('/content/restored_images', exist_ok=True)

#weather restoration function
def simple_weather_restoration(input_image_path, output_path):
    img = cv2.imread(input_image_path)
    if img is None:
        print(f"Error reading image: {input_image_path}")
        return None

    #convert to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l = clahe.apply(l)

    # Merge the CLAHE enhanced L-channel back with A and B channels
    lab = cv2.merge((l, a, b))

    # Convert back to BGR color space
    enhanced_img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # Apply bilateral filter to reduce noise while preserving edges
    restored_img = cv2.bilateralFilter(enhanced_img, 9, 75, 75)

    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save the restored image
    cv2.imwrite(output_path, restored_img)

    return output_path

# Process all weather conditions
print("Starting to process all test images across all weather conditions...")

# Process each weather condition
for weather in weather_folders:
    print(f"\nProcessing {weather} conditions...")

    # Create weather-specific directory
    os.makedirs(f'/content/restored_images/{weather}', exist_ok=True)

    # Get all images for this weather
    images_dir = os.path.join(base_path, weather, 'images')
    image_files = glob.glob(os.path.join(images_dir, '*.jpg')) + glob.glob(os.path.join(images_dir, '*.png'))

    # Process each image
    for image_path in tqdm(image_files):
        image_filename = os.path.basename(image_path)
        base_name, ext = os.path.splitext(image_filename)

        # Restore the image
        restored_path = os.path.join(f'/content/restored_images/{weather}', f"{base_name}_restored{ext}")
        simple_weather_restoration(image_path, restored_path)

print("\nAll weather images have been restored!")
print("Restored images are saved in /content/restored_images/")

# Print summary of processed images
for weather in weather_folders:
    restored_dir = f'/content/restored_images/{weather}'
    num_restored = len(glob.glob(os.path.join(restored_dir, '*.*')))
    print(f"- {weather}: {num_restored} images restored")

Starting to process all test images across all weather conditions...

Processing Fog conditions...


100%|██████████| 300/300 [01:05<00:00,  4.55it/s]



Processing Rain conditions...


100%|██████████| 200/200 [00:54<00:00,  3.66it/s]



Processing Snow conditions...


100%|██████████| 204/204 [00:45<00:00,  4.46it/s]



Processing Sand conditions...


100%|██████████| 323/323 [01:06<00:00,  4.89it/s]


All weather images have been restored!
Restored images are saved in /content/restored_images/
- Fog: 300 images restored
- Rain: 200 images restored
- Snow: 204 images restored
- Sand: 323 images restored


In [24]:
cd yolov5/

/content/yolov5


In [25]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt \
               --data data_test_fog.yaml \
               --img 640 \
               --task test \
               --project results_after_weather_restoration \
               --name test_fog


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [32]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt \
               --data data_test_rain.yaml \
               --img 640 \
               --task test \
               --project results_after_weather_restoration \
               --name test_rain


val: data=data_test_rain.yaml, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results_after_weather_restoration, name=test_rain, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Rain/labels.cache... 200 images, 0 backgrounds, 0 corrupt: 100% 200/200 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [01:32<00:00, 13.18s/it]
                   all        200       1560     0.0777      0.441     0.0629     0.0323
                person        200       1560     0.0777      0.441     0.0629     0.0323
Speed: 7

In [33]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt \
               --data data_test_sand.yaml \
               --img 640 \
               --task test \
               --project results_after_weather_restoration \
               --name test_sand


val: data=data_test_sand.yaml, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results_after_weather_restoration, name=test_sand, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Sand/labels.cache... 323 images, 0 backgrounds, 0 corrupt: 100% 323/323 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 11/11 [02:35<00:00, 14.16s/it]
                   all        323       2052      0.138      0.343      0.107     0.0651
                person        323       2052      0.138      0.343      0.107     0.0651
Speed:

In [34]:
!python val.py --weights /content/yolov5/runs/train/exp3/weights/best.pt \
               --data data_test_snow.yaml \
               --img 640 \
               --task test \
               --project results_after_weather_restoration \
               --name test_snow


val: data=data_test_snow.yaml, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=results_after_weather_restoration, name=test_snow, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/cse527/dawn/Snow/labels.cache... 204 images, 0 backgrounds, 0 corrupt: 100% 204/204 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [01:48<00:00, 15.44s/it]
                   all        204       1913     0.0532      0.393     0.0427     0.0222
                person        204       1913     0.0532      0.393     0.0427     0.0222
Speed: 5

In [35]:
!python detect.py --weights /content/yolov5/runs/train/exp5/weights/best.pt\
                  --img 640 \
                  --source /content/restored_images/Rain/rain_storm-001_restored.jpg \
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp5/weights/best.pt'], source=/content/restored_images/Rain/rain_storm-001_restored.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/restored_images/Rain/rain_storm-001_restored.jpg: 448x640 2 vehicles, 318.0ms
Speed: 2.6ms pre-process, 318.0ms inference, 1.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp14
1 labels saved to runs/detect/exp14/labels


In [36]:
!python detect.py --weights /content/yolov5/runs/train/exp5/weights/best.pt\
                  --img 640 \
                  --source /content/restored_images/Rain/rain_storm-002_restored.jpg \
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp5/weights/best.pt'], source=/content/restored_images/Rain/rain_storm-002_restored.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/restored_images/Rain/rain_storm-002_restored.jpg: 416x640 4 persons, 2 vehicles, 325.8ms
Speed: 3.4ms pre-process, 325.8ms inference, 1.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp15
1 labels saved to runs/detect/exp15/labels


In [37]:
!python detect.py --weights /content/yolov5/runs/train/exp5/weights/best.pt\
                  --img 640 \
                  --source /content/restored_images/Rain/rain_storm-003_restored.jpg \
                  --conf 0.25 \
                  --save-txt \
                  --save-conf


detect: weights=['/content/yolov5/runs/train/exp5/weights/best.pt'], source=/content/restored_images/Rain/rain_storm-003_restored.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/restored_images/Rain/rain_storm-003_restored.jpg: 352x640 1 person, 379.6ms
Speed: 2.6ms pre-process, 379.6ms inference, 1.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp16
1 labels saved to runs/detect/exp16/labels
